In [1]:
import os
import cv2
import numpy as np

from tqdm import tqdm
from utils import (
    overlay_image_with_blend, 
    gaussian_kernel, zip_folder)

from utils.algorithms import (
    perlin_noise, 
    generate_parallel_or_perpendicular_lines)

In [2]:
def generate_images(shape_image_base=(512, 512), gaussian_size_low=4, gaussian_size_high=40, num_lines_range=(2, 6)):
    """
    Generate images with random parallel or perpendicular lines and apply Gaussian blur and Perlin noise.

    Parameters:
    shape_image_base (tuple): Base shape of the image (height, width).
    gaussian_size_low (int): Minimum size of the Gaussian kernel.
    gaussian_size_high (int): Maximum size of the Gaussian kernel.
    num_lines_range (tuple): Range for the number of lines to generate (min, max).

    Returns:
    tuple: Generated images (img, img_background, img_pretil_binary, img_pretil).
    """
    img = np.zeros(shape_image_base)
    img_tmp = np.zeros(shape_image_base)
    img_pretil = np.zeros(shape_image_base)
    img_pretil_binary = np.zeros(shape_image_base)

    # Generate between 2 and 6 parallel or perpendicular lines
    num_lines = np.random.randint(num_lines_range[0], num_lines_range[1] + 1)
    lines = generate_parallel_or_perpendicular_lines(shape_image_base, num_lines)
    
    # Overlay the lines on the image
    for line in lines:
        img_line = np.zeros(shape_image_base)
        for point in line:
            random_size = np.random.randint(gaussian_size_low, gaussian_size_high)
            gaussian = gaussian_kernel(random_size)
            img_line += overlay_image_with_blend(img_tmp, gaussian, position=point)
        img_line = img_line / np.max(img_line)    
        img = np.maximum(img, img_line)
    
    img_pretil = img.copy()
    img_pretil_binary = np.where(img >= 0.1, 1, 0)
    img_background = 1 - img_pretil_binary  
    img += perlin_noise((512, 512, 1), octaves=(0.01, 0.02, 0.04, 0.08, 0.1, 0.2, 0.3, 1)) * np.random.uniform(0.1, 0.3)
    
    return img, img_background, img_pretil_binary, img_pretil

In [3]:
num_images = 10
dataset_path = 'dataset'

if not os.path.exists(dataset_path):
    os.makedirs(dataset_path)
else:
    raise FileExistsError(f'Folder {dataset_path} already exists. Please remove it before running this script.')

gsd = 0.1  # Ground Sampling Distance (m/px)
shape_image_base = (512, 512)
pretril_size_low = np.round(0.4 / gsd)
pretil_size_high = np.round(4 / gsd)

for idx in tqdm(range(num_images)):
    img, img_backgound, img_pretil_binary, img_pretil = generate_images(shape_image_base,pretril_size_low, pretil_size_high)
    img_backgound = (img_backgound * 255).astype(np.uint8)
    img_pretil_binary = (img_pretil_binary * 255).astype(np.uint8)
    img_pretil = (img_pretil * 255).astype(np.uint8)
    
    np.savez_compressed(f'{dataset_path}/img_{idx}.npz', img)
    cv2.imwrite(f'{dataset_path}/img_backgound_{idx}.png', img_backgound)
    cv2.imwrite(f'{dataset_path}/img_pretil_binary_{idx}.png', img_pretil_binary)
    cv2.imwrite(f'{dataset_path}/img_pretil_{idx}.png', img_pretil)




100%|██████████| 10/10 [00:06<00:00,  1.65it/s]


In [4]:
zip_folder(dataset_path, f'dataset.zip')

Zipping files: 100%|██████████| 40/40 [00:00<00:00, 141.97file/s]
